In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
N = 1000
series = np.sin(0.1 * np.arange(N))
#plt.plot(series)

[0.         0.84147098 0.90929743]


In [15]:
X = []
Y = []
T = 10
for t in range(N - T):
  x = series[t: t + T]
  y = series[t + T]
  X.append(x)
  Y.append(y)

X = np.array(X).reshape(-1, T)
Y = np.array(Y).reshape(-1, 1)


(990, 1)

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Assuming series is your time series data
series = np.array([...])  # Your time series data

# Hyperparameters
T = 10  # Number of time steps to look back
input_size = 1  # Assuming univariate time series
output_size = 1  # Assuming single value prediction
hidden_size = 32
learning_rate = 0.001
num_epochs = 100

# Prepare data
X = []
Y = []
N = len(series)
for t in range(N - T):
    x = series[t: t + T]
    y = series[t + T]
    X.append(x)
    Y.append(y)

X = np.array(X).reshape(-1, T, input_size)
Y = np.array(Y).reshape(-1, output_size)

# Convert to PyTorch tensors
X_train = torch.from_numpy(X.astype(np.float32))
Y_train = torch.from_numpy(Y.astype(np.float32))

# Define the model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_cell = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0):
        out, hn = self.rnn_cell(x, h0)
        out = out[:, -1, :]  # Take output from last time step
        out = self.fc(out)
        return out

# Instantiate the model
model = SimpleRNN(input_size, hidden_size, output_size)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train, torch.zeros(1, X_train.size(0), hidden_size))
    loss = criterion(outputs, Y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Prediction
future = 10  # Number of future time steps to predict
preds = []
x = X_train[-1]  # Taking the last input sequence from training data
for _ in range(future):
    with torch.no_grad():
        x = x.reshape(1, T, input_size)
        output = model(x, torch.zeros(1, 1, hidden_size))
        preds.append(output.item())
        x = np.append(x[:, 1:, :], output.numpy().reshape(1, 1, input_size), axis=1)

# Plotting predictions
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(np.arange(len(series)), series, label='Original')
plt.plot(np.arange(len(series), len(series) + future), preds, label='Predicted')
plt.legend()
plt.show()
